In [ ]:
import snowflake.snowpark as snowpark
from snowflake.snowpark.functions import col

def fetch_ivy_league_data(session: snowpark.Session) -> snowpark.DataFrame:
    import requests
    import pandas as pd
    
    COLLEGE_API_KEY = "YOUR_API_KEY_HERE"  # Replace with your actual College Scorecard API key
    COLLEGE_BASE_URL = "https://api.data.gov/ed/collegescorecard/v1/schools"
    
    # Ivy League schools with their coordinates
    IVY_LEAGUE_SCHOOLS = {
        "Columbia University": {"lat": 40.807536, "lon": -73.962573, "city": "New York", "state": "NY"},
        "Brown University": {"lat": 41.82399, "lon": -71.412834, "city": "Providence", "state": "RI"},
        "Cornell University": {"lat": 42.440498, "lon": -76.495697, "city": "Ithaca", "state": "NY"},
        "Dartmouth College": {"lat": 43.700859, "lon": -72.289398, "city": "Hanover", "state": "NH"},
        "Harvard University": {"lat": 42.3656347, "lon": -71.1040018, "city": "Cambridge", "state": "MA"},
        "Princeton University": {"lat": 40.3496953, "lon": -74.6597376, "city": "Princeton", "state": "NJ"},
        "University of Pennsylvania": {"lat": 39.9527237, "lon": -75.1635262, "city": "Philadelphia", "state": "PA"},
        "Yale University": {"lat": 41.3082138, "lon": -72.9250518, "city": "New Haven", "state": "CT"}
    }
    
    # Get school enrollment data from College Scorecard API
    fields = ["school.name", "school.city", "school.state", "latest.student.size"]
    school_data = {}
    
    for school_name in IVY_LEAGUE_SCHOOLS.keys():
        params = {
            "api_key": COLLEGE_API_KEY,
            "school.name": school_name,
            "fields": ",".join(fields),
            "per_page": 1
        }
        
        try:
            response = requests.get(COLLEGE_BASE_URL, params=params)
            response.raise_for_status()
            data = response.json()
            
            if data["results"]:
                record = data["results"][0]
                school_data[school_name] = {
                    "enrollment": record.get("latest", {}).get("student", {}).get("size", None)
                }
        except Exception as e:
            school_data[school_name] = {"enrollment": None}
    
    # Get weather data from Open-Meteo API
    weather_url = "https://api.open-meteo.com/v1/forecast"
    
    lats = [info["lat"] for info in IVY_LEAGUE_SCHOOLS.values()]
    lons = [info["lon"] for info in IVY_LEAGUE_SCHOOLS.values()]
    
    weather_params = {
        "latitude": lats,
        "longitude": lons,
        "daily": "temperature_2m_min",
        "hourly": ["temperature_2m", "sunshine_duration"],
        "current": ["precipitation", "snowfall"],
        "timezone": "America/New_York",
        "wind_speed_unit": "mph",
        "temperature_unit": "fahrenheit",
        "precipitation_unit": "inch",
        "start_date": "2026-01-01",
        "end_date": "2026-01-31",
    }
    
    weather_response = requests.get(weather_url, params=weather_params)
    weather_data = weather_response.json()
    
    # Combine all data
    results = []
    school_names = list(IVY_LEAGUE_SCHOOLS.keys())
    
    for i, school_name in enumerate(school_names):
        school_info = IVY_LEAGUE_SCHOOLS[school_name]
        loc_weather = weather_data[i]
        
        current = loc_weather.get('current', {})
        current_snowfall = current.get('snowfall', 0)
        current_precipitation = current.get('precipitation', 0)
        
        # Calculate average temperature from daily data
        daily = loc_weather.get('daily', {})
        min_temps = daily.get('temperature_2m_min', [])
        avg_min_temp = sum(min_temps) / len(min_temps) if min_temps else None
        
        # Calculate total sunshine hours
        hourly = loc_weather.get('hourly', {})
        sunshine = hourly.get('sunshine_duration', [])
        total_sunshine_hours = sum([s/3600 for s in sunshine if s]) if sunshine else 0
        
        results.append({
            "SCHOOL": school_name,
            "CITY": school_info["city"],
            "STATE": school_info["state"],
            "LATITUDE": school_info["lat"],
            "LONGITUDE": school_info["lon"],
            "ENROLLMENT": school_data.get(school_name, {}).get("enrollment"),
            "CURRENT_SNOWFALL_INCHES": current_snowfall,
            "CURRENT_PRECIPITATION_INCHES": current_precipitation,
            "AVG_MIN_TEMP_F_JAN": round(avg_min_temp, 1) if avg_min_temp else None,
            "TOTAL_SUNSHINE_HOURS_JAN": round(total_sunshine_hours, 1),
            "ELEVATION_M": loc_weather.get('elevation', 0)
        })
    
    # Convert to pandas DataFrame and sort by snowfall
    pandas_df = pd.DataFrame(results)
    pandas_df = pandas_df.sort_values('CURRENT_SNOWFALL_INCHES', ascending=False)
    
    # Add snow rank
    pandas_df.insert(0, 'SNOW_RANK', range(1, len(pandas_df) + 1))
    
    # Convert to Snowpark DataFrame
    return session.create_dataframe(pandas_df)

In [ ]:
CREATE OR REPLACE PROCEDURE fetch_ivy_league_snow_data()
RETURNS TABLE(
    SNOW_RANK NUMBER,
    SCHOOL VARCHAR,
    CITY VARCHAR,
    STATE VARCHAR,
    LATITUDE FLOAT,
    LONGITUDE FLOAT,
    ENROLLMENT NUMBER,
    CURRENT_SNOWFALL_INCHES FLOAT,
    CURRENT_PRECIPITATION_INCHES FLOAT,
    AVG_MIN_TEMP_F_JAN FLOAT,
    TOTAL_SUNSHINE_HOURS_JAN FLOAT,
    ELEVATION_M FLOAT
)
LANGUAGE PYTHON
RUNTIME_VERSION = '3.8'
PACKAGES = ('snowflake-snowpark-python', 'requests', 'pandas')
HANDLER = 'fetch_ivy_league_data';

-- Run it and create a table
CREATE OR REPLACE TABLE ivy_league_snowfall_weather AS
SELECT * FROM TABLE(fetch_ivy_league_snow_data());

-- View results ranked by current snowfall
SELECT 
    SNOW_RANK,
    SCHOOL,
    CITY,
    STATE,
    CURRENT_SNOWFALL_INCHES,
    AVG_MIN_TEMP_F_JAN,
    ENROLLMENT
FROM ivy_league_snowfall_weather
ORDER BY SNOW_RANK;